In [ ]:
import os
import numpy as np
import torch

#import torchvision.transforms as transforms
from torch import nn
from torchvision import datasets
import transforms as T
#from torchvision import transforms

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
use_cuda = torch.cuda.is_available()
print(use_cuda)

In [ ]:
act = nn.Sigmoid()

In [ ]:
#from torchvision.transforms import functional as F

In [ ]:
mytforms = []
mytforms.append(T.RandomCrop(224))
mytforms.append(T.RandomHorizontalFlip(0.5))
mytforms.append(T.RandomVerticalFlip(0.5))
mytforms.append(T.ToTensor())
mytforms = T.Compose(mytforms)

In [ ]:
# for debugging purposes
trainset = datasets.VOCSegmentation(
    root='/scratch/ssd/cciw/', year='2012', image_set='train', download=False, 
    transforms=T.ToTensor())
    #, transforms=mytforms)

In [ ]:
bs = 10
trainloader = torch.utils.data.DataLoader(trainset, batch_size=bs, shuffle=False)

In [ ]:
for inputs, targets in trainloader:
    inputs, targets = inputs.cuda(), targets.cuda()
    break
    #inputs, targets = inputs.cuda(), targets.cuda()
print(inputs.shape)
print(targets.shape)

In [ ]:
#loss_fn(pred, targets.unsqueeze(1).float())

In [ ]:
idx = 2
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
inputs_nhwc = inputs.permute(0, 2, 3, 1)
ax1.imshow(inputs_nhwc.detach().cpu().numpy()[idx])
ax2.imshow(targets.detach().cpu().numpy()[idx])

In [ ]:
#np.unique(targets.detach().cpu().numpy()[1, 0])

In [ ]:
%matplotlib inline

In [ ]:
#1/255

In [ ]:
#from torchvision.models.segmentation import fcn_resnet50

In [ ]:
#net = fcn_resnet50(num_classes=1).cuda()

In [ ]:
from torchvision.models import segmentation as models

In [ ]:
model_names = sorted(name for name in models.__dict__
    if name.islower() and not name.startswith("__")
    and callable(models.__dict__[name]))
print(model_names)

In [ ]:
resume = '/scratch/gallowaa/cciw/logs/v0.1.0/checkpoint/fcn_resnet50_bs40_wd5e-04_aug1_1.ckpt'

print('==> Resuming from checkpoint..')
checkpoint = torch.load(resume)
net = checkpoint['net']
best_acc = checkpoint['loss']
start_epoch = checkpoint['epoch'] + 1
torch.set_rng_state(checkpoint['rng_state'])

In [ ]:
start_epoch

In [ ]:
#net = models.__dict__['fcn_resnet50'](num_classes=1)
#net = net.cuda()

In [ ]:
#net.eval()
net.train()
print(net.training)

In [ ]:
pred = act(net(inputs)['out'])
pred.shape

In [ ]:
pred.max()

In [ ]:
inputs_nhwc = inputs.permute(0, 2, 3, 1)

In [ ]:
#idx = 2 # train mode
for idx in range(bs):
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(9, 3))
    ax1.imshow(inputs_nhwc.detach().cpu().numpy()[idx])
    ax2.imshow(pred.detach().cpu().numpy()[idx, 0])
    ax3.imshow(targets.detach().cpu().numpy()[idx])
    ax1.set_title('Image'); ax2.set_title('Logits'); ax3.set_title('Labels')
    pretty_image([ax1, ax2, ax3])
    plt.savefig(resume.split('/')[-1].split('.')[0] + '_demo_preds_train' + str(net.training) + '_%d.png' % idx)

In [ ]:
def pretty_image(axes):
    for ax in axes:
        ax.axis('off')
    plt.tight_layout()

In [ ]:
loss_fn = nn.BCEWithLogitsLoss()

In [ ]:
optimizer = torch.optim.SGD(net.parameters(), lr=0.1, weight_decay=5e-4)

In [ ]:
epochs = 1
for epoch in range(epochs):
    for batch, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.cuda(), targets.cuda()
        pred = net(inputs)['out']
        
        loss = loss_fn(pred, 
                       torch.LongTensor(
                           targets.unsqueeze(1) * 255)
                      )
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print('Epoch [{}/{}], Loss: {:.4f}'
          .format(epoch + 1, epochs, loss.item()))

In [ ]:
#pixel_acc(targets, targets)
pixel_acc(pred, pred)

In [ ]:
targets.shape

In [ ]:
def pixel_acc(pred, label):
    #_, preds = torch.max(pred, dim=1)
    preds = torch.argmax(pred, dim=1)
    valid = (label >= 0).long()
    acc_sum = torch.sum(valid * (preds == label).long())
    pixel_sum = torch.sum(valid)
    acc = acc_sum.float() / (pixel_sum.float() + 1e-10)
    return acc

In [ ]:
class SegmentationModuleBase(nn.Module):
    def __init__(self):
        super(SegmentationModuleBase, self).__init__()

    def pixel_acc(self, pred, label):
        _, preds = torch.max(pred, dim=1)
        valid = (label >= 0).long()
        acc_sum = torch.sum(valid * (preds == label).long())
        pixel_sum = torch.sum(valid)
        acc = acc_sum.float() / (pixel_sum.float() + 1e-10)
        return acc


class SegmentationModule(SegmentationModuleBase):
    def __init__(self, net_enc, net_dec, crit, deep_sup_scale=None):
        super(SegmentationModule, self).__init__()
        self.encoder = net_enc
        self.decoder = net_dec
        self.crit = crit
        self.deep_sup_scale = deep_sup_scale

    def forward(self, feed_dict, *, segSize=None):
        # training
        if segSize is None:
            if self.deep_sup_scale is not None: # use deep supervision technique
                (pred, pred_deepsup) = self.decoder(self.encoder(feed_dict['img_data'], return_feature_maps=True))
            else:
                pred = self.decoder(self.encoder(feed_dict['img_data'], return_feature_maps=True))

            loss = self.crit(pred, feed_dict['seg_label'])
            if self.deep_sup_scale is not None:
                loss_deepsup = self.crit(pred_deepsup, feed_dict['seg_label'])
                loss = loss + loss_deepsup * self.deep_sup_scale

            acc = self.pixel_acc(pred, feed_dict['seg_label'])
            return loss, acc
        # inference
        else:
            pred = self.decoder(self.encoder(feed_dict['img_data'], return_feature_maps=True), segSize=segSize)
            return pred